In [ ]:
import requests
import os
import concurrent.futures

### Parallel batch-continuous version

In [ ]:
def fetch_page(page):
    url = f'{api_url}?bbox={bbox}&page={page}'
    print(f'Sending HTTP request to {url}...')
    response = requests.get(url)
    return page, response

In [ ]:
# TODO: partition the overall bbox in smaller bboxes such that the API request will be accepted by OSM.
# bbox_NY = '-74.259,40.477,-73.700,40.918' # Example: New York City bounding box
bbox = '-74.25909,40.477399,-74.150181,40.516178'
limit_deg = 0.25

# OSM API endpoint for GPS trackpoints
api_url = 'https://api.openstreetmap.org/api/0.6/trackpoints'

# Directory to save the downloaded GPX files
output_dir = 'gpx_traces'
os.makedirs(output_dir, exist_ok=True)

max_workers = 24
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    next_page = 0
    # Submit an initial batch of tasks equal to max_workers
    future_to_page = {executor.submit(fetch_page, p): p for p in range(next_page, next_page + max_workers)}
    next_page += max_workers
    keep_going = True
    while future_to_page:
        # Iterate over completed futures
        for future in concurrent.futures.as_completed(list(future_to_page.keys())):
            p, response = future.result()
            if response.status_code == 200 and '<trkpt' in response.text:
                gpx_filename = os.path.join(output_dir, f'trackpoints_page_{p}.gpx')
                with open(gpx_filename, 'w', encoding='utf-8') as file:
                    file.write(response.text)
                print(f'Saved: {gpx_filename}')
                
                # Submit a new task to keep the pool full.
                future_to_page[executor.submit(fetch_page, next_page)] = next_page
                next_page += 1
            else:
                print(f'No more GPX data available for page {p}.')
                keep_going = False
                
            # Remove the completed future from the dictionary.
            del future_to_page[future]